<a href="https://colab.research.google.com/github/yashs087/hate_speech_detection/blob/master/Hate_speech_detection2(21022020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
#mport pickle
#import sys
from sklearn.feature_extraction.text import TfidfVectorizer #vectorizer to change and test
import nltk #natural language tool kit
from nltk.stem.porter import *
import string
import re #regular expression
#from textstat.textstat import *
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [0]:
data= pd.read_csv("/content/dataset2.csv")
data.head()

,Label,Comment
0,1,"""You fuck your dad."""
1,0,"""i really don't understand your point.\xa0 It ..."
2,0,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,"""listen if you dont wanna get married to a man..."
4,0,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [0]:
len(data)

3947

In [0]:
set(data["Label"])

{0, 1}

In [0]:
list(data["Label"]).count(0)

2898

In [0]:
list(data["Label"]).count(1)

1049

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
tweets = data.Comment

stopwords=stopwords = nltk.corpus.stopwords.words("english")  #removing stopwords for better performance

stemmer = PorterStemmer() #algorithm for for removing the common morphological and inflexional endings from words in English.

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split() if t not in stopwords and len(t)> 2]
    tokens = [t for t in tokens if len(t)>2]
    return tokens

In [0]:
vectorizer = TfidfVectorizer( #An alternative is to calculate word frequencies, and by far the most popular method is called TF-IDF
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3), #an n-gram is a contiguous sequence of n items from a given sample of text
    stop_words=stopwords,
    use_idf=True,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.75
    )

In [0]:
tfidf=vectorizer.fit_transform(tweets).toarray()

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
x = tfidf
y = data['Label'].astype(int)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.35)

In [0]:
classifiers = [
    LogisticRegression(),
    LinearSVC(),
    GaussianNB(),
    SGDClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier()
]

In [0]:
def df(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [0]:
Results=[]
for i,item in enumerate(classifiers):
    clf = item
    clf.fit(X_train, y_train)
    print(item,'\n')
    y_preds = clf.predict(X_test)
    report = classification_report( y_test, y_preds,output_dict=True)
    Results.append(report)
    print(report,"\n")
    print(confusion_matrix(y_test, y_preds),"\n\n\n")


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 

{'0': {'precision': 0.8519607843137255, 'recall': 0.8544739429695182, 'f1-score': 0.8532155130093275, 'support': 1017}, '1': {'precision': 0.5911602209944752, 'recall': 0.5863013698630137, 'f1-score': 0.5887207702888584, 'support': 365}, 'accuracy': 0.7836468885672938, 'macro avg': {'precision': 0.7215605026541003, 'recall': 0.7203876564162659, 'f1-score': 0.720968141649093, 'support': 1382}, 'weighted avg': {'precision': 0.7830807513097265, 'recall': 0.7836468885672938, 'f1-score': 0.7833598103371342, 'support': 1382}} 

[[869 148]
 [151 214]] 



LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



GaussianNB(priors=None, var_smoothing=1e-09) 

{'0': {'precision': 0.797768479776848, 'recall': 0.5624385447394297, 'f1-score': 0.6597462514417531, 'support': 1017}, '1': {'precision': 0.3308270676691729, 'recall': 0.6027397260273972, 'f1-score': 0.42718446601941745, 'support': 365}, 'accuracy': 0.573082489146165, 'macro avg': {'precision': 0.5642977737230104, 'recall': 0.5825891353834134, 'f1-score': 0.5434653587305853, 'support': 1382}, 'weighted avg': {'precision': 0.6744445901825633, 'recall': 0.573082489146165, 'f1-score': 0.598324361659443, 'support': 1382}} 

[[572 445]
 [145 220]] 



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verb

In [0]:
lr=pd.DataFrame(Results[0]).transpose()
svc=pd.DataFrame(Results[1]).transpose()
gnb=pd.DataFrame(Results[2]).transpose()
sgd=pd.DataFrame(Results[3]).transpose()
knn=pd.DataFrame(Results[4]).transpose()
tree=pd.DataFrame(Results[5]).transpose()
forest=pd.DataFrame(Results[6]).transpose()
mlp=pd.DataFrame(Results[7]).transpose()

In [0]:
result = pd.concat([lr,svc,gnb,sgd,knn,tree,forest,mlp], keys=['LogisticRegression','LinearSVC','GaussianNB','SGDClassifier','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','MLPClassifier'])

In [0]:
result

precision    recall  f1-score      support
LogisticRegression     0              0.851961  0.854474  0.853216  1017.000000
                       1              0.591160  0.586301  0.588721   365.000000
                       accuracy       0.783647  0.783647  0.783647     0.783647
                       macro avg      0.721561  0.720388  0.720968  1382.000000
                       weighted avg   0.783081  0.783647  0.783360  1382.000000
LinearSVC              0              0.843750  0.796460  0.819423  1017.000000
                       1              0.509479  0.589041  0.546379   365.000000
                       accuracy       0.741679  0.741679  0.741679     0.741679
                       macro avg      0.676614  0.692751  0.682901  1382.000000
                       weighted avg   0.755466  0.741679  0.747310  1382.000000
GaussianNB             0              0.797768  0.562439  0.659746  1017.000000
                       1              0.330827  0.602740  0.427184   365.000000
                       accuracy       0.573082  0.573082  0.573082     0.573082
                       macro avg      0.564298  0.582589  0.543465  1382.000000
                       weighted avg   0.674445  0.573082  0.598324  1382.000000
SGDClassifier          0              0.845998  0.821042  0.833333  1017.000000
                       1              0.539241  0.583562  0.560526   365.000000
                       accuracy       0.758321  0.758321  0.758321     0.758321
                       macro avg      0.692619  0.702302  0.696930  1382.000000
                       weighted avg   0.764980  0.758321  0.761282  1382.000000
KNeighborsClassifier   0              0.783961  0.941986  0.855739  1017.000000
                       1              0.631250  0.276712  0.384762   365.000000
                       accuracy       0.766281  0.766281  0.766281     0.766281
                       macro avg      0.707605  0.609349  0.620251  1382.000000
                       weighted avg   0.743628  0.766281  0.731349  1382.000000
DecisionTreeClassifier 0              0.853125  0.805310  0.828528  1017.000000
                       1              0.530806  0.613699  0.569250   365.000000
                       accuracy       0.754703  0.754703  0.754703     0.754703
                       macro avg      0.691965  0.709504  0.698889  1382.000000
                       weighted avg   0.767997  0.754703  0.760050  1382.000000
RandomForestClassifier 0              0.858365  0.887906  0.872885  1017.000000
                       1              0.654545  0.591781  0.621583   365.000000
                       accuracy       0.809696  0.809696  0.809696     0.809696
                       macro avg      0.756455  0.739843  0.747234  1382.000000
                       weighted avg   0.804534  0.809696  0.806514  1382.000000
MLPClassifier          0              0.844614  0.871190  0.857696  1017.000000
                       1              0.606607  0.553425  0.578797   365.000000
                       accuracy       0.787265  0.787265  0.787265     0.787265
                       macro avg      0.725610  0.712307  0.718246  1382.000000
                       weighted avg   0.781754  0.787265  0.784036  1382.000000